# 1 - Importation
## 1.1 - Librairies

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
#pd.options.mode.chained_assignment = 'warn'

## 1.2 - Données

In [2]:
geo = pd.read_excel("geo.xlsx")
population = pd.read_excel("population.xlsx")
dispo = pd.read_excel("dispo.xlsx")
conso_poulet = pd.read_excel("consommation_poulet.xlsx")
inflation_nourriture_1517 = pd.read_excel("inflation_nourriture_1517.xlsx")
PIB_habitant = pd.read_excel("PIB_habitant.xlsx")
cout_production = pd.read_excel("cout_production.xlsx")
cout_douanes = pd.read_excel("cout_conformite_importation.xlsx")
abordabilite_nourriture = pd.read_excel("abordabilite_nourriture.xlsx")
delai_douanes = pd.read_excel("delai_importation_frontiere.xlsx")
produits_poulet = pd.read_excel("produits_poulet.xlsx")
echanges_poulet_france = pd.read_excel("echanges_poulet_france.xlsx")
GES_poulet = pd.read_excel("GES_poulet.xlsx")
stabilite = pd.read_excel("stabilite.xlsx")
dispo_habitant = pd.read_excel("dispo_alimentaire_habitant.xlsx")

# 2 - Regroupement population
## 2.1 - df population
### 2.1.1 - Lecture du df

In [3]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564 entries, 0 to 2563
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code_pays        2564 non-null   object 
 1   pays             2564 non-null   object 
 2   annee            2564 non-null   int64  
 3   population*1000  2564 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 80.2+ KB


In [4]:
population.head()

,code_pays,pays,annee,population*1000
0,AFG,Afghanistan,2007,25903.301
1,AFG,Afghanistan,2008,26427.199
2,AFG,Afghanistan,2009,27385.307
3,AFG,Afghanistan,2010,28189.672
4,AFG,Afghanistan,2011,29249.157


### 2.1.2 - Création du df temporaire "population2017"

In [5]:
population2017 = population.loc[population["annee"] == 2017, :]
population2017["population*1000"] = round(population2017["population*1000"]*1000,0).astype(int)

In [6]:
population2017.head()

,code_pays,pays,annee,population*1000
10,AFG,Afghanistan,2017,35643418
21,ZAF,Afrique du Sud,2017,56641209
32,ALB,Albanie,2017,2879355
43,DZA,Algérie,2017,41136546
54,DEU,Allemagne,2017,82624374


In [7]:
population = population.astype({'annee':str})

### 2.1.3 - Création du df pop_evo (2007 - 2017) qui indiquera l'évolution de la population entre 2007 et 2017

In [8]:
# Sélection des années 2007 et 2017
pop1 = population.loc[population["annee"].isin(["2007","2017"])].reset_index(drop=True)
pop1.head()

,code_pays,pays,annee,population*1000
0,AFG,Afghanistan,2007,25903.301
1,AFG,Afghanistan,2017,35643.418
2,ZAF,Afrique du Sud,2007,49996.094
3,ZAF,Afrique du Sud,2017,56641.209
4,ALB,Albanie,2007,2976.084


### 2.1.4 - Pivotement du df pour passer les valeurs de la série "annee" en colonnes "2007" et "2017"

In [9]:
pop_evo = pop1.pivot(index= "pays",columns="annee", values="population*1000")
pop_evo.head()

annee,2007,2017
pays,,
Afghanistan,25903.301,35643.418
Afrique du Sud,49996.094,56641.209
Albanie,2976.084,2879.355
Algérie,33983.827,41136.546
Allemagne,81183.830,82624.374


### 2.1.5 - Création de la colonne "%_evolution_2007/2017", arrondie à 2 décimales et suppression des colonnes "2007" et "2017"

In [10]:
pop_evo["%_evolution_2007/2017"] =round((pop_evo["2017"]-pop_evo["2007"])*100 / pop_evo["2007"],2) 
pop_evo.drop(columns=["2007","2017"], inplace = True)
pop_evo.head()

annee,%_evolution_2007/2017
pays,
Afghanistan,37.60
Afrique du Sud,13.29
Albanie,-3.25
Algérie,21.05
Allemagne,1.77


### 2.1.6 - Merge des df "population2017" et "pop_evo" en un df "population"

In [11]:
population = pd.merge(population2017,pop_evo, how = "inner", on = "pays" )
population = population[["pays","code_pays","population*1000","%_evolution_2007/2017"]]
population.head()

,pays,code_pays,population*1000,%_evolution_2007/2017
0,Afghanistan,AFG,35643418,37.60
1,Afrique du Sud,ZAF,56641209,13.29
2,Albanie,ALB,2879355,-3.25
3,Algérie,DZA,41136546,21.05
4,Allemagne,DEU,82624374,1.77


### 2.1.7 - Renommage de la colonne "population\*1000"

In [12]:
population.rename(columns={
    "population*1000" : "population"
},inplace = True)
population.head()

,pays,code_pays,population,%_evolution_2007/2017
0,Afghanistan,AFG,35643418,37.60
1,Afrique du Sud,ZAF,56641209,13.29
2,Albanie,ALB,2879355,-3.25
3,Algérie,DZA,41136546,21.05
4,Allemagne,DEU,82624374,1.77


In [13]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pays                   235 non-null    object 
 1   code_pays              235 non-null    object 
 2   population             235 non-null    int32  
 3   %_evolution_2007/2017  228 non-null    float64
dtypes: float64(1), int32(1), object(2)
memory usage: 6.6+ KB


## 2.2 - df abordabilite_nourriture
### 2.2.1 - Lecture du df

In [14]:
abordabilite_nourriture.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_pays                   169 non-null    object 
 1   pays                        169 non-null    object 
 2   cout_viande_pers/jour(USD)  169 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.1+ KB


In [15]:
abordabilite_nourriture.head()

,code_pays,pays,cout_viande_pers/jour(USD)
0,ZAF,Afrique du Sud,1.059
1,ALB,Albanie,1.204
2,DZA,Algérie,0.964
3,DEU,Allemagne,0.666
4,AGO,Angola,1.011


### 2.2.2 - merge des df "population" et "abordabilite_nourriture" en df "population"

In [16]:
population = pd.merge(population,abordabilite_nourriture, on="pays", how="outer")
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   pays                        235 non-null    object 
 1   code_pays_x                 235 non-null    object 
 2   population                  235 non-null    int32  
 3   %_evolution_2007/2017       228 non-null    float64
 4   code_pays_y                 169 non-null    object 
 5   cout_viande_pers/jour(USD)  169 non-null    float64
dtypes: float64(2), int32(1), object(3)
memory usage: 10.2+ KB


### 2.2.3 - Sélection des colonnes à conserver + renommage si nécessaire

In [17]:
population = population[["pays","code_pays_x","population","%_evolution_2007/2017","cout_viande_pers/jour(USD)"]]
population.rename(columns={"code_pays_x" : "code_pays"},inplace=True)
population.head()

,pays,code_pays,population,%_evolution_2007/2017,cout_viande_pers/jour(USD)
0,Afghanistan,AFG,35643418,37.60,NaN
1,Afrique du Sud,ZAF,56641209,13.29,1.059
2,Albanie,ALB,2879355,-3.25,1.204
3,Algérie,DZA,41136546,21.05,0.964
4,Allemagne,DEU,82624374,1.77,0.666


### 2.2.4 - conversion de la colonne "cout_viande_pers/jour(USD)" par an + renommage

In [18]:
population["cout_viande_pers/jour(USD)"] =round(population["cout_viande_pers/jour(USD)"]*365,2) 
population.rename(columns={"cout_viande_pers/jour(USD)" : "cout_viande_pers/an(USD)" },inplace=True)

In [19]:
population.head()

,pays,code_pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD)
0,Afghanistan,AFG,35643418,37.60,NaN
1,Afrique du Sud,ZAF,56641209,13.29,386.54
2,Albanie,ALB,2879355,-3.25,439.46
3,Algérie,DZA,41136546,21.05,351.86
4,Allemagne,DEU,82624374,1.77,243.09


## 2.3 - df PIB_habitant
### 2.3.1 - Lecture du df

In [20]:
PIB_habitant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_pays          187 non-null    object 
 1   pays               187 non-null    object 
 2   PIB_habitant(USD)  187 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.5+ KB


In [21]:
PIB_habitant.head()

,code_pays,pays,PIB_habitant(USD)
0,NLD,Pays-Bas (Royaume des),55088.6
1,AFG,Afghanistan,2058.4
2,ZAF,Afrique du Sud,13860.3
3,ALB,Albanie,12771.0
4,DZA,Algérie,11737.4


### 2.3.2 - Merge des df "population" et "PIB_habitant" en df "population"

In [22]:
population = pd.merge(population,PIB_habitant, on="code_pays",how="outer")

### 2.3.3 - Sélection des colonnes à conserver + renommage si nécessaire

In [23]:
population = population[["pays_x","code_pays","population","%_evolution_2007/2017","cout_viande_pers/an(USD)","PIB_habitant(USD)"]]
population.rename(columns={"pays_x" : "pays"}, inplace=True)

In [24]:
population.head()

,pays,code_pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD)
0,Afghanistan,AFG,35643418,37.60,NaN,2058.4
1,Afrique du Sud,ZAF,56641209,13.29,386.54,13860.3
2,Albanie,ALB,2879355,-3.25,439.46,12771.0
3,Algérie,DZA,41136546,21.05,351.86,11737.4
4,Allemagne,DEU,82624374,1.77,243.09,53071.5


## 2.4 - df inflation_nourriture_1517
### 2.4.1 - Lecture df

In [25]:
inflation_nourriture_1517.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2436 entries, 0 to 2435
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   code_pays     2436 non-null   object 
 1   pays          2436 non-null   object 
 2   mois          2436 non-null   object 
 3   Tx_inflation  2436 non-null   float64
dtypes: float64(1), object(3)
memory usage: 76.2+ KB


In [26]:
inflation_nourriture_1517.head()

,code_pays,pays,mois,Tx_inflation
0,NLD,Pays-Bas (Royaume des),Janvier,1.504363
1,NLD,Pays-Bas (Royaume des),Février,2.104010
2,NLD,Pays-Bas (Royaume des),Mars,1.879885
3,NLD,Pays-Bas (Royaume des),Avril,1.818002
4,NLD,Pays-Bas (Royaume des),Mai,2.447207


### 2.4.2 - Calcul tu taux d'inflation sur l'année

In [27]:
def calcul_inflation_annuelle(inflation):
    resultat_inflation = pd.DataFrame(columns=["pays","Tx_inflation"])
    
    pays_uniques = inflation["pays"].unique()
    
    for pays in pays_uniques:
        inflation_pays = inflation[inflation["pays"] == pays]
        Tx_inflation = round(inflation_pays["Tx_inflation"].rolling(window=12).mean().dropna().values[-1],2)
        resultat_inflation = pd.concat([resultat_inflation, pd.DataFrame({"pays": [pays], "Tx_inflation": [Tx_inflation]})], ignore_index=True)
    
    return resultat_inflation

In [28]:
inflation1517 = calcul_inflation_annuelle(inflation_nourriture_1517)
inflation1517.head()

,pays,Tx_inflation
0,Pays-Bas (Royaume des),2.66
1,Afghanistan,6.96
2,Afrique du Sud,6.90
3,Albanie,3.92
4,Algérie,5.04


### 2.4.3 - Merge des df "population" et "inflation1517" en df "population"

In [29]:
population = pd.merge(population, inflation1517, on="pays", how="outer")

In [30]:
pd.options.display.float_format = '{:,.0f}'.format
colonnes_a_formater = ["population","%_evolution_2007/2017","cout_viande_pers/an(USD)","PIB_habitant(USD)","Tx_inflation"]
for colonne in colonnes_a_formater:
    population[colonne] = population[colonne].fillna(0).astype(float)


In [31]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   pays                      236 non-null    object 
 1   code_pays                 235 non-null    object 
 2   population                236 non-null    float64
 3   %_evolution_2007/2017     236 non-null    float64
 4   cout_viande_pers/an(USD)  236 non-null    float64
 5   PIB_habitant(USD)         236 non-null    float64
 6   Tx_inflation              236 non-null    float64
dtypes: float64(5), object(2)
memory usage: 13.0+ KB


In [32]:
population.head()

,pays,code_pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD),Tx_inflation
0,Afghanistan,AFG,"35,643,418",38,0,"2,058",7
1,Afrique du Sud,ZAF,"56,641,209",13,387,"13,860",7
2,Albanie,ALB,"2,879,355",-3,439,"12,771",4
3,Algérie,DZA,"41,136,546",21,352,"11,737",5
4,Allemagne,DEU,"82,624,374",2,243,"53,072",3


## 2.5 - df dispo_habitant
### 2.5.1 - Lecture df

In [33]:
dispo_habitant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   pays                            180 non-null    object 
 1   code_pays                       180 non-null    object 
 2   dispo_alimentaire(g/pers/jour)  180 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.3+ KB


In [34]:
dispo_habitant.head()

,pays,code_pays,dispo_alimentaire(g/pers/jour)
0,Afghanistan,AFG,5
1,Afrique du Sud,ZAF,99
2,Albanie,ALB,29
3,Algérie,DZA,17
4,Allemagne,DEU,30


### 2.5.2 - Renommage de la colonne "dispo_alimentaire(g/pers/jour)" en "dispo_alimentaire(kg/pers/an)"

In [35]:
dispo_habitant.rename(columns={"dispo_alimentaire(g/pers/jour)" : "dispo_alimentaire(kg/pers/an)"}, inplace = True)

### 2.5.3 - passage des valeurs de "dispo_alimentaire(kg/pers/an)" en kg/pers/an

In [36]:
dispo_habitant["dispo_alimentaire(kg/pers/an)"] = round((dispo_habitant["dispo_alimentaire(kg/pers/an)"]/1000)*365,2)

In [37]:
dispo_habitant.head()

,pays,code_pays,dispo_alimentaire(kg/pers/an)
0,Afghanistan,AFG,2
1,Afrique du Sud,ZAF,36
2,Albanie,ALB,11
3,Algérie,DZA,6
4,Allemagne,DEU,11


## 2.6 - Création du df définitif "population" par le merge des df "population" et "dispo_habitant"

In [38]:
population = pd.merge(population, dispo_habitant, on="pays", how ="outer")
population.drop("code_pays_y", axis = 1 , inplace = True)
population.rename(columns={"code_pays_x":"code_pays"}, inplace= True)

In [39]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           236 non-null    object 
 1   code_pays                      235 non-null    object 
 2   population                     236 non-null    float64
 3   %_evolution_2007/2017          236 non-null    float64
 4   cout_viande_pers/an(USD)       236 non-null    float64
 5   PIB_habitant(USD)              236 non-null    float64
 6   Tx_inflation                   236 non-null    float64
 7   dispo_alimentaire(kg/pers/an)  180 non-null    float64
dtypes: float64(6), object(2)
memory usage: 14.9+ KB


### 2.6.1 - Observation des pays ayant des valeurs nulles dans la colonne "dispo_alimentaire(kg/pers/an)"

In [40]:
dispo_null = population[population["dispo_alimentaire(kg/pers/an)"].isnull()]
pays_null = dispo_null["pays"].tolist()
print(pays_null)

['Andorre', 'Anguilla', 'Aruba', 'Bahreïn', 'Bermudes', 'Bhoutan', 'Bonaire, Saint-Eustache et Saba', 'Brunéi Darussalam', 'Curaçao', 'Érythrée', 'Gibraltar', 'Groenland', 'Guadeloupe', 'Guam', 'Guinée équatoriale', 'Guyane française', 'Île de Man', 'Îles Caïmanes', 'Îles Cook', 'Îles Falkland (Malvinas)', 'Îles Féroé', 'Îles Mariannes du Nord', 'Îles Marshall', 'Îles Turques-et-Caïques', 'Îles Vierges américaines', 'Îles Vierges britanniques', 'Îles Wallis-et-Futuna', 'Liechtenstein', 'Martinique', 'Mayotte', 'Micronésie (États fédérés de)', 'Monaco', 'Montserrat', 'Nauru', 'Nioué', 'Palaos', 'Palestine', 'Porto Rico', 'Qatar', 'Réunion', 'Sahara occidental', 'Saint-Barthélemy', 'Sainte-Hélène, Ascension et Tristan da Cunha', 'Saint-Marin', 'Saint-Martin (partie française)', 'Saint-Pierre-et-Miquelon', 'Saint-Siège', 'Samoa américaines', 'Singapour', 'Sint Maarten (partie néerlandaise)', 'Somalie', 'Soudan du Sud', 'Tokélaou', 'Tonga', 'Tuvalu', "Îles d'Åland"]


### 2.6.2 - Passage des valeurs nulles à "0.0"

In [41]:
population["dispo_alimentaire(kg/pers/an)"] = population["dispo_alimentaire(kg/pers/an)"].fillna(0).astype(float)

In [42]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           236 non-null    object 
 1   code_pays                      235 non-null    object 
 2   population                     236 non-null    float64
 3   %_evolution_2007/2017          236 non-null    float64
 4   cout_viande_pers/an(USD)       236 non-null    float64
 5   PIB_habitant(USD)              236 non-null    float64
 6   Tx_inflation                   236 non-null    float64
 7   dispo_alimentaire(kg/pers/an)  236 non-null    float64
dtypes: float64(6), object(2)
memory usage: 14.9+ KB


# 3 - Regroupement produit_poulet
## 3.1 - df dispo
### 3.1.1 - Lecture du df

In [43]:
dispo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code_pays  840 non-null    object 
 1   pays       840 non-null    object 
 2   element    840 non-null    object 
 3   tonnes     840 non-null    float64
dtypes: float64(1), object(3)
memory usage: 26.4+ KB


In [44]:
dispo.head()

,code_pays,pays,element,tonnes
0,AFG,Afghanistan,Production,"27,850"
1,AFG,Afghanistan,Importations - Quantité,"28,937"
2,AFG,Afghanistan,Variation de stock,"-6,727"
3,AFG,Afghanistan,Disponibilité alimentaire en quantité (tonnes),"63,514"
4,ZAF,Afrique du Sud,Production,"1,658,159"


### 3.1.2 - Renommage de la colonne 'tonnes" en "valeur"

In [45]:
dispo.rename(columns={"tonnes" : "valeur"},inplace=True)

In [46]:
dispo.head()

,code_pays,pays,element,valeur
0,AFG,Afghanistan,Production,"27,850"
1,AFG,Afghanistan,Importations - Quantité,"28,937"
2,AFG,Afghanistan,Variation de stock,"-6,727"
3,AFG,Afghanistan,Disponibilité alimentaire en quantité (tonnes),"63,514"
4,ZAF,Afrique du Sud,Production,"1,658,159"


### 3.1.3 - Pivotement du df pour passer les valeurs de la série "element" en colonnes

In [47]:
dispo = dispo.pivot(index="pays",columns="element",values="valeur").reset_index()
dispo = dispo.merge(produits_poulet[["pays","code_pays"]], on="pays",how="left")
dispo = dispo.drop_duplicates()
dispo.head()

,pays,Disponibilité alimentaire en quantité (tonnes),Exportations - Quantité,Importations - Quantité,Production,Variation de stock,code_pays
0,Afghanistan,"63,514",NaN,"28,937","27,850","-6,727",AFG
4,Afrique du Sud,"2,070,000","58,259","480,346","1,658,159","5,700",ZAF
8,Albanie,"30,192",NaN,"17,280","12,912",0,ALB
10,Algérie,"262,965",NaN,0,"262,965",0,DZA
12,Allemagne,"913,829","300,561","474,951","1,009,565","181,293",DEU


### 3.1.4 - Renommage des colonnes

In [48]:
dispo.rename(columns={
    "Disponibilité alimentaire en quantité (tonnes)" : "disponibilite_interieure",
    "Exportations - Quantité" : "export",
    "Importations - Quantité" : "import",
    "Production" : "production",
    "Variation de stock" : "variation_stock"
},inplace = True)
dispo.head()

,pays,disponibilite_interieure,export,import,production,variation_stock,code_pays
0,Afghanistan,"63,514",NaN,"28,937","27,850","-6,727",AFG
4,Afrique du Sud,"2,070,000","58,259","480,346","1,658,159","5,700",ZAF
8,Albanie,"30,192",NaN,"17,280","12,912",0,ALB
10,Algérie,"262,965",NaN,0,"262,965",0,DZA
12,Allemagne,"913,829","300,561","474,951","1,009,565","181,293",DEU


## 3.2 - df conso_poulet
### 3.2.1 - Lecture du df

In [49]:
conso_poulet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   code_pays           180 non-null    object
 1   pays                180 non-null    object
 2   conso_poulet*1000t  180 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.3+ KB


In [50]:
conso_poulet.head()

,code_pays,pays,conso_poulet*1000t
0,NLD,Pays-Bas (Royaume des),273
1,AFG,Afghanistan,64
2,ZAF,Afrique du Sud,2110
3,ALB,Albanie,33
4,DZA,Algérie,286


### 3.2.2 - passage des valeurs de la colonne "conso_poulet\*1000t" en unités, puis renommage en "nourriture"

In [51]:
conso_poulet["conso_poulet*1000t"] = conso_poulet["conso_poulet*1000t"]*1000
conso_poulet.rename(columns={"conso_poulet*1000t" : "nourriture"},inplace=True)
conso_poulet.head()

,code_pays,pays,nourriture
0,NLD,Pays-Bas (Royaume des),273000
1,AFG,Afghanistan,64000
2,ZAF,Afrique du Sud,2110000
3,ALB,Albanie,33000
4,DZA,Algérie,286000


### 3.2.3 - merge des df "dispo" et "conso_poulet" en df "dispo"

In [52]:
dispo = pd.merge(dispo, conso_poulet, on="pays", how="inner")
dispo.head()

,pays,disponibilite_interieure,export,import,production,variation_stock,code_pays_x,code_pays_y,nourriture
0,Afghanistan,"63,514",NaN,"28,937","27,850","-6,727",AFG,AFG,64000
1,Afrique du Sud,"2,070,000","58,259","480,346","1,658,159","5,700",ZAF,ZAF,2110000
2,Albanie,"30,192",NaN,"17,280","12,912",0,ALB,ALB,33000
3,Algérie,"262,965",NaN,0,"262,965",0,DZA,DZA,286000
4,Allemagne,"913,829","300,561","474,951","1,009,565","181,293",DEU,DEU,1492000


### 3.2.4 - Modification de l'ordre d'affichage des colonnes

In [53]:
dispo = dispo[["pays","code_pays_x","production","import","export","variation_stock","nourriture","disponibilite_interieure"]]
dispo.rename(columns={"code_pays_x":"code_pays"},inplace=True)
dispo.head()

,pays,code_pays,production,import,export,variation_stock,nourriture,disponibilite_interieure
0,Afghanistan,AFG,"27,850","28,937",NaN,"-6,727",64000,"63,514"
1,Afrique du Sud,ZAF,"1,658,159","480,346","58,259","5,700",2110000,"2,070,000"
2,Albanie,ALB,"12,912","17,280",NaN,0,33000,"30,192"
3,Algérie,DZA,"262,965",0,NaN,0,286000,"262,965"
4,Allemagne,DEU,"1,009,565","474,951","300,561","181,293",1492000,"913,829"


## 3.3 - df produits_poulet
### 3.3.1 - Lecture du df

In [54]:
produits_poulet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code_pays  640 non-null    object 
 1   pays       640 non-null    object 
 2   element    640 non-null    object 
 3   unite      640 non-null    object 
 4   valeur     640 non-null    float64
dtypes: float64(1), object(4)
memory usage: 25.1+ KB


In [55]:
produits_poulet.head()

,code_pays,pays,element,unite,valeur
0,AFG,Afghanistan,Importations - Quantité,tonnes,"28,937"
1,AFG,Afghanistan,Importations - Valeur,1000 US$,"28,934"
2,AFG,Afghanistan,Exportations - Quantité,tonnes,37
3,AFG,Afghanistan,Exportations - Valeur,1000 US$,31
4,ZAF,Afrique du Sud,Importations - Quantité,tonnes,"480,346"


### 3.3.2 - Pivotement du df pour passer les valeurs de la série "element" en colonnes

In [56]:
produits_poulet = produits_poulet.pivot(index="pays",columns="element",values="valeur").reset_index()
produits_poulet = produits_poulet.merge(dispo[["pays","code_pays"]], on="pays",how="left")
produits_poulet = produits_poulet.drop_duplicates()
produits_poulet.head()

,pays,Exportations - Quantité,Exportations - Valeur,Importations - Quantité,Importations - Valeur,code_pays
0,Afghanistan,37,31,"28,937","28,934",AFG
1,Afrique du Sud,"58,259","87,683","480,346","410,502",ZAF
2,Albanie,NaN,NaN,"17,515","16,481",ALB
3,Algérie,NaN,NaN,0,2,DZA
4,Allemagne,"300,561","629,582","474,951","1,101,592",DEU


### 3.3.3 - Renommage des colonnes

In [57]:
produits_poulet.rename(columns={
    "Exportations - Quantité" : "export_tonnes",
    "Exportations - Valeur" : "export_USD",
    "Importations - Quantité" : "import_tonnes",
    "Importations - Valeur" : "import_USD"
},inplace = True)
produits_poulet.head()

,pays,export_tonnes,export_USD,import_tonnes,import_USD,code_pays
0,Afghanistan,37,31,"28,937","28,934",AFG
1,Afrique du Sud,"58,259","87,683","480,346","410,502",ZAF
2,Albanie,NaN,NaN,"17,515","16,481",ALB
3,Algérie,NaN,NaN,0,2,DZA
4,Allemagne,"300,561","629,582","474,951","1,101,592",DEU


### 3.3.4 - Passage des colonnes relatives aux prix en unités, remplacement des valeurs nulles par 0 et modification du type en int

In [58]:
produits_poulet["export_USD"] = produits_poulet["export_USD"]*1000
produits_poulet["export_USD"] = produits_poulet["export_USD"].fillna(0)
produits_poulet["export_USD"] = produits_poulet["export_USD"].astype(np.int64)
produits_poulet["import_USD"] = produits_poulet["import_USD"]*1000
produits_poulet["import_USD"] = produits_poulet["import_USD"].fillna(0)
produits_poulet["import_USD"] = produits_poulet["import_USD"].astype(np.int64)
produits_poulet.head()

,pays,export_tonnes,export_USD,import_tonnes,import_USD,code_pays
0,Afghanistan,37,31000,"28,937",28934000,AFG
1,Afrique du Sud,"58,259",87683000,"480,346",410502000,ZAF
2,Albanie,NaN,0,"17,515",16481000,ALB
3,Algérie,NaN,0,0,2000,DZA
4,Allemagne,"300,561",629582000,"474,951",1101592000,DEU


### 3.3.5 - Création des colonnes de prix à la tonne pour les importations et exportations

In [59]:
produits_poulet["prix_export_USD/tonne"] = round(produits_poulet["export_USD"]/produits_poulet["export_tonnes"],2).fillna(0)
produits_poulet["prix_import_USD/tonne"] = round(produits_poulet["import_USD"]/produits_poulet["import_tonnes"],2).fillna(0)
produits_poulet.head()

,pays,export_tonnes,export_USD,import_tonnes,import_USD,code_pays,prix_export_USD/tonne,prix_import_USD/tonne
0,Afghanistan,37,31000,"28,937",28934000,AFG,838,"1,000"
1,Afrique du Sud,"58,259",87683000,"480,346",410502000,ZAF,"1,505",855
2,Albanie,NaN,0,"17,515",16481000,ALB,0,941
3,Algérie,NaN,0,0,2000,DZA,0,"5,000"
4,Allemagne,"300,561",629582000,"474,951",1101592000,DEU,"2,095","2,319"


### 3.3.6 - Création du df "dispo_poulet" par merge des df "dispo" et "produits_poulet"

In [60]:
dispo_poulet = pd.merge(dispo, produits_poulet, on="pays", how="inner")
dispo_poulet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   pays                      178 non-null    object 
 1   code_pays_x               178 non-null    object 
 2   production                176 non-null    float64
 3   import                    174 non-null    float64
 4   export                    128 non-null    float64
 5   variation_stock           178 non-null    float64
 6   nourriture                178 non-null    int64  
 7   disponibilite_interieure  178 non-null    float64
 8   export_tonnes             126 non-null    float64
 9   export_USD                178 non-null    int64  
 10  import_tonnes             175 non-null    float64
 11  import_USD                178 non-null    int64  
 12  code_pays_y               178 non-null    object 
 13  prix_export_USD/tonne     178 non-null    float64
 14  prix_impor

### 3.3.7 - Choix des colonnes à conserver et modification de leur ordre d'affichage

In [61]:
dispo_poulet = dispo_poulet[[
    "pays",
    "code_pays_x",
    "production",
    "import_tonnes",
    "import_USD",
    "prix_import_USD/tonne",
    "export_tonnes",
    "export_USD",
    "prix_export_USD/tonne",
    "variation_stock",
    "nourriture",
    "disponibilite_interieure",
]]
dispo_poulet.rename(columns={"code_pays_x":"code_pays"},inplace=True)
dispo_poulet.head()

,pays,code_pays,production,import_tonnes,import_USD,prix_import_USD/tonne,export_tonnes,export_USD,prix_export_USD/tonne,variation_stock,nourriture,disponibilite_interieure
0,Afghanistan,AFG,"27,850","28,937",28934000,"1,000",37,31000,838,"-6,727",64000,"63,514"
1,Afrique du Sud,ZAF,"1,658,159","480,346",410502000,855,"58,259",87683000,"1,505","5,700",2110000,"2,070,000"
2,Albanie,ALB,"12,912","17,515",16481000,941,NaN,0,0,0,33000,"30,192"
3,Algérie,DZA,"262,965",0,2000,"5,000",NaN,0,0,0,286000,"262,965"
4,Allemagne,DEU,"1,009,565","474,951",1101592000,"2,319","300,561",629582000,"2,095","181,293",1492000,"913,829"


### 3.3.8 - Insertion de séparateurs de milliers

In [62]:
pd.options.display.float_format = '{:,.0f}'.format
colonnes_a_formater = ["production","import_tonnes","import_USD","prix_import_USD/tonne","export_tonnes","export_USD",
                       "prix_export_USD/tonne","nourriture","variation_stock","disponibilite_interieure"]
for colonne in colonnes_a_formater:
    dispo_poulet[colonne] = dispo_poulet[colonne].fillna(0).astype(float)
dispo_poulet.head()

,pays,code_pays,production,import_tonnes,import_USD,prix_import_USD/tonne,export_tonnes,export_USD,prix_export_USD/tonne,variation_stock,nourriture,disponibilite_interieure
0,Afghanistan,AFG,"27,850","28,937","28,934,000","1,000",37,"31,000",838,"-6,727","64,000","63,514"
1,Afrique du Sud,ZAF,"1,658,159","480,346","410,502,000",855,"58,259","87,683,000","1,505","5,700","2,110,000","2,070,000"
2,Albanie,ALB,"12,912","17,515","16,481,000",941,0,0,0,0,"33,000","30,192"
3,Algérie,DZA,"262,965",0,"2,000","5,000",0,0,0,0,"286,000","262,965"
4,Allemagne,DEU,"1,009,565","474,951","1,101,592,000","2,319","300,561","629,582,000","2,095","181,293","1,492,000","913,829"


## 3.4 - df echange_poulet_france
### 3.4.1 - Lecture du df

In [63]:
echanges_poulet_france.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code_pays  256 non-null    object 
 1   pays       256 non-null    object 
 2   element    256 non-null    object 
 3   unite      256 non-null    object 
 4   valeur     256 non-null    float64
dtypes: float64(1), object(4)
memory usage: 10.1+ KB


In [64]:
echanges_poulet_france.head()

,code_pays,pays,element,unite,valeur
0,ZAF,Afrique du Sud,Importations - Quantité,tonnes,0
1,ZAF,Afrique du Sud,Importations - Valeur,1000 US$,2
2,DEU,Allemagne,Importations - Quantité,tonnes,"19,245"
3,DEU,Allemagne,Importations - Valeur,1000 US$,"69,354"
4,DEU,Allemagne,Exportations - Quantité,tonnes,"32,492"


### 3.4.2 - Pivotement du df pour passer les valeurs de la série "element" en colonnes

In [65]:
echanges_poulet_france = echanges_poulet_france.pivot(index="pays",columns="element",values="valeur").reset_index()
echanges_poulet_france = echanges_poulet_france.merge(dispo[["pays","code_pays"]], on="pays",how="left")
echanges_poulet_france = echanges_poulet_france.drop_duplicates()
echanges_poulet_france.head()

,pays,Exportations - Quantité,Exportations - Valeur,Importations - Quantité,Importations - Valeur,code_pays
0,Afrique du Sud,NaN,NaN,0,2,ZAF
1,Allemagne,"32,492","94,163","19,245","69,354",DEU
2,Angola,NaN,NaN,203,284,AGO
3,Arabie saoudite,NaN,NaN,"97,485","141,541",SAU
4,Arménie,NaN,NaN,816,630,ARM


In [66]:
# Passage des valeurs d'importation et d'exportation en unité
echanges_poulet_france["Exportations - Valeur"] = echanges_poulet_france["Exportations - Valeur"]*1000
echanges_poulet_france["Importations - Valeur"] = echanges_poulet_france["Importations - Valeur"]*1000

### 3.4.3 - Renommage des colonnes

In [67]:
echanges_poulet_france.rename(columns={
    "Exportations - Quantité" : "exportFrance_tonnes",
    "Exportations - Valeur" : "exportFrance_USD",
    "Importations - Quantité" : "importFrance_tonnes",
    "Importations - Valeur" : "importFrance_USD"
},inplace = True)
echanges_poulet_france.head()

,pays,exportFrance_tonnes,exportFrance_USD,importFrance_tonnes,importFrance_USD,code_pays
0,Afrique du Sud,NaN,NaN,0,"2,000",ZAF
1,Allemagne,"32,492","94,163,000","19,245","69,354,000",DEU
2,Angola,NaN,NaN,203,"284,000",AGO
3,Arabie saoudite,NaN,NaN,"97,485","141,541,000",SAU
4,Arménie,NaN,NaN,816,"630,000",ARM


### 3.4.4 -Création des colonnes de prix à la tonne pour les importations et exportations, des excédents commerciaux

In [68]:
echanges_poulet_france["prix_exportFrance_USD/tonne"] = round(echanges_poulet_france["exportFrance_USD"]/echanges_poulet_france["exportFrance_tonnes"],2).fillna(0)
echanges_poulet_france["prix_importFrance_USD/tonne"] = round(echanges_poulet_france["importFrance_USD"]/echanges_poulet_france["importFrance_tonnes"],2).fillna(0)
echanges_poulet_france["excedent_commercial_fr_tonnes"] = echanges_poulet_france["exportFrance_tonnes"] - echanges_poulet_france["importFrance_tonnes"]
echanges_poulet_france["excedent_commercial_fr_USD"] = echanges_poulet_france["exportFrance_USD"] - echanges_poulet_france["importFrance_USD"]
echanges_poulet_france.head()

,pays,exportFrance_tonnes,exportFrance_USD,importFrance_tonnes,importFrance_USD,code_pays,prix_exportFrance_USD/tonne,prix_importFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD
0,Afrique du Sud,NaN,NaN,0,"2,000",ZAF,0,"25,000",NaN,NaN
1,Allemagne,"32,492","94,163,000","19,245","69,354,000",DEU,"2,898","3,604","13,247","24,809,000"
2,Angola,NaN,NaN,203,"284,000",AGO,0,"1,397",NaN,NaN
3,Arabie saoudite,NaN,NaN,"97,485","141,541,000",SAU,0,"1,452",NaN,NaN
4,Arménie,NaN,NaN,816,"630,000",ARM,0,773,NaN,NaN


### 3.4.5 - Modification de l'ordre d'affichage des colonnes

In [69]:
echanges_poulet_france = echanges_poulet_france[[
    "pays",
    "code_pays",
    "exportFrance_tonnes",
    "exportFrance_USD",
    "prix_exportFrance_USD/tonne",
    "importFrance_tonnes",
    "importFrance_USD",
    "prix_importFrance_USD/tonne",
    "excedent_commercial_fr_tonnes",
    "excedent_commercial_fr_USD"
]]

In [70]:
echanges_poulet_france.head()

,pays,code_pays,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,importFrance_tonnes,importFrance_USD,prix_importFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD
0,Afrique du Sud,ZAF,NaN,NaN,0,0,"2,000","25,000",NaN,NaN
1,Allemagne,DEU,"32,492","94,163,000","2,898","19,245","69,354,000","3,604","13,247","24,809,000"
2,Angola,AGO,NaN,NaN,0,203,"284,000","1,397",NaN,NaN
3,Arabie saoudite,SAU,NaN,NaN,0,"97,485","141,541,000","1,452",NaN,NaN
4,Arménie,ARM,NaN,NaN,0,816,"630,000",773,NaN,NaN


###  3.4.6 - Création du df "produit poulet (merge des df "dispo_poulet" et "echanges_poulet_france"

In [71]:
produit_poulet = pd.merge(dispo_poulet,echanges_poulet_france, on="pays",how="outer")
produit_poulet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           182 non-null    object 
 1   code_pays_x                    178 non-null    object 
 2   production                     178 non-null    float64
 3   import_tonnes                  178 non-null    float64
 4   import_USD                     178 non-null    float64
 5   prix_import_USD/tonne          178 non-null    float64
 6   export_tonnes                  178 non-null    float64
 7   export_USD                     178 non-null    float64
 8   prix_export_USD/tonne          178 non-null    float64
 9   variation_stock                178 non-null    float64
 10  nourriture                     178 non-null    float64
 11  disponibilite_interieure       178 non-null    float64
 12  code_pays_y                    96 non-null     obj

In [72]:
produit_poulet.head()

,pays,code_pays_x,production,import_tonnes,import_USD,prix_import_USD/tonne,export_tonnes,export_USD,prix_export_USD/tonne,variation_stock,...,disponibilite_interieure,code_pays_y,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,importFrance_tonnes,importFrance_USD,prix_importFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD
0,Afghanistan,AFG,"27,850","28,937","28,934,000","1,000",37,"31,000",838,"-6,727",...,"63,514",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afrique du Sud,ZAF,"1,658,159","480,346","410,502,000",855,"58,259","87,683,000","1,505","5,700",...,"2,070,000",ZAF,NaN,NaN,0,0,"2,000","25,000",NaN,NaN
2,Albanie,ALB,"12,912","17,515","16,481,000",941,0,0,0,0,...,"30,192",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algérie,DZA,"262,965",0,"2,000","5,000",0,0,0,0,...,"262,965",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Allemagne,DEU,"1,009,565","474,951","1,101,592,000","2,319","300,561","629,582,000","2,095","181,293",...,"913,829",DEU,"32,492","94,163,000","2,898","19,245","69,354,000","3,604","13,247","24,809,000"


### 3.4.7 - Choix des colonnes à conserver et modification de leur ordre d'affichage

In [73]:
produit_poulet = produit_poulet[[
    "pays",
    "code_pays_x",
    "production",
    "import_tonnes",
    "import_USD",
    "prix_import_USD/tonne",
    "importFrance_tonnes",
    "importFrance_USD",
    "prix_importFrance_USD/tonne",
    "export_tonnes",
    "export_USD",
    "prix_export_USD/tonne",
    "exportFrance_tonnes",
    "exportFrance_USD",
    "prix_exportFrance_USD/tonne",
    "excedent_commercial_fr_tonnes",
    "excedent_commercial_fr_USD",
    "variation_stock",
    "nourriture",
    "disponibilite_interieure"
]]

In [74]:
produit_poulet.rename(columns={"code_pays_x" : "code_pays"},inplace=True)
produit_poulet.head()

,pays,code_pays,production,import_tonnes,import_USD,prix_import_USD/tonne,importFrance_tonnes,importFrance_USD,prix_importFrance_USD/tonne,export_tonnes,export_USD,prix_export_USD/tonne,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure
0,Afghanistan,AFG,"27,850","28,937","28,934,000","1,000",NaN,NaN,NaN,37,"31,000",838,NaN,NaN,NaN,NaN,NaN,"-6,727","64,000","63,514"
1,Afrique du Sud,ZAF,"1,658,159","480,346","410,502,000",855,0,"2,000","25,000","58,259","87,683,000","1,505",NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000"
2,Albanie,ALB,"12,912","17,515","16,481,000",941,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0,"33,000","30,192"
3,Algérie,DZA,"262,965",0,"2,000","5,000",NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0,"286,000","262,965"
4,Allemagne,DEU,"1,009,565","474,951","1,101,592,000","2,319","19,245","69,354,000","3,604","300,561","629,582,000","2,095","32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829"


In [75]:
produit_poulet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           182 non-null    object 
 1   code_pays                      178 non-null    object 
 2   production                     178 non-null    float64
 3   import_tonnes                  178 non-null    float64
 4   import_USD                     178 non-null    float64
 5   prix_import_USD/tonne          178 non-null    float64
 6   importFrance_tonnes            95 non-null     float64
 7   importFrance_USD               95 non-null     float64
 8   prix_importFrance_USD/tonne    100 non-null    float64
 9   export_tonnes                  178 non-null    float64
 10  export_USD                     178 non-null    float64
 11  prix_export_USD/tonne          178 non-null    float64
 12  exportFrance_tonnes            33 non-null     flo

## 3.5 - df cout_production
### 3.5.1 - Lecture df

In [76]:
cout_production.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   code_pays               44 non-null     object 
 1   pays                    44 non-null     object 
 2   cout_production(USD/t)  44 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.2+ KB


In [77]:
cout_production.head()

,code_pays,pays,cout_production(USD/t)
0,ZAF,Afrique du Sud,"2,017"
1,DZA,Algérie,"2,181"
2,DEU,Allemagne,"1,460"
3,ARG,Argentine,"1,324"
4,ARM,Arménie,"3,741"


### 3.5.2 - Merge des df "produit_poulet" et "cout_production"

In [78]:
produit_poulet = pd.merge(produit_poulet, cout_production, on="pays", how="left")
produit_poulet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           182 non-null    object 
 1   code_pays_x                    178 non-null    object 
 2   production                     178 non-null    float64
 3   import_tonnes                  178 non-null    float64
 4   import_USD                     178 non-null    float64
 5   prix_import_USD/tonne          178 non-null    float64
 6   importFrance_tonnes            95 non-null     float64
 7   importFrance_USD               95 non-null     float64
 8   prix_importFrance_USD/tonne    100 non-null    float64
 9   export_tonnes                  178 non-null    float64
 10  export_USD                     178 non-null    float64
 11  prix_export_USD/tonne          178 non-null    float64
 12  exportFrance_tonnes            33 non-null     flo

In [79]:
produit_poulet.head()

,pays,code_pays_x,production,import_tonnes,import_USD,prix_import_USD/tonne,importFrance_tonnes,importFrance_USD,prix_importFrance_USD/tonne,export_tonnes,...,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,code_pays_y,cout_production(USD/t)
0,Afghanistan,AFG,"27,850","28,937","28,934,000","1,000",NaN,NaN,NaN,37,...,NaN,NaN,NaN,NaN,NaN,"-6,727","64,000","63,514",NaN,NaN
1,Afrique du Sud,ZAF,"1,658,159","480,346","410,502,000",855,0,"2,000","25,000","58,259",...,NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000",ZAF,"2,017"
2,Albanie,ALB,"12,912","17,515","16,481,000",941,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,0,"33,000","30,192",NaN,NaN
3,Algérie,DZA,"262,965",0,"2,000","5,000",NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,0,"286,000","262,965",DZA,"2,181"
4,Allemagne,DEU,"1,009,565","474,951","1,101,592,000","2,319","19,245","69,354,000","3,604","300,561",...,"32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829",DEU,"1,460"


### 3.5.3 - Choix des colonnes à conserver et modification de leur ordre d'affichage

In [80]:
produit_poulet = produit_poulet[[
    "pays",
    "code_pays_x",
    "production",
    "cout_production(USD/t)",
    "import_tonnes",
    "import_USD",
    "prix_import_USD/tonne",
    "importFrance_tonnes",
    "importFrance_USD",
    "prix_importFrance_USD/tonne",
    "export_tonnes",
    "export_USD",
    "prix_export_USD/tonne",
    "exportFrance_tonnes",
    "exportFrance_USD",
    "prix_exportFrance_USD/tonne",
    "excedent_commercial_fr_tonnes",
    "excedent_commercial_fr_USD",
    "variation_stock",
    "nourriture",
    "disponibilite_interieure",
]]
produit_poulet.rename(columns={"code_pays_x" : "code_pays"},inplace=True)
produit_poulet.head()

,pays,code_pays,production,cout_production(USD/t),import_tonnes,import_USD,prix_import_USD/tonne,importFrance_tonnes,importFrance_USD,prix_importFrance_USD/tonne,...,export_USD,prix_export_USD/tonne,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure
0,Afghanistan,AFG,"27,850",NaN,"28,937","28,934,000","1,000",NaN,NaN,NaN,...,"31,000",838,NaN,NaN,NaN,NaN,NaN,"-6,727","64,000","63,514"
1,Afrique du Sud,ZAF,"1,658,159","2,017","480,346","410,502,000",855,0,"2,000","25,000",...,"87,683,000","1,505",NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000"
2,Albanie,ALB,"12,912",NaN,"17,515","16,481,000",941,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0,"33,000","30,192"
3,Algérie,DZA,"262,965","2,181",0,"2,000","5,000",NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0,"286,000","262,965"
4,Allemagne,DEU,"1,009,565","1,460","474,951","1,101,592,000","2,319","19,245","69,354,000","3,604",...,"629,582,000","2,095","32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829"


## 3.6 - df GES_poulet
### 3.6.1 - Lecture df

In [81]:
GES_poulet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code_pays  596 non-null    object 
 1   pays       596 non-null    object 
 2   element    596 non-null    object 
 3   unite      596 non-null    object 
 4   valeur     596 non-null    float64
dtypes: float64(1), object(4)
memory usage: 23.4+ KB


In [82]:
GES_poulet.head()

,code_pays,pays,element,unite,valeur
0,AFG,Afghanistan,tonne_co2/tonne_poulet,kg CO2eq/kg,0
1,AFG,Afghanistan,emission_co2(tonnes),kilotonnes,10
2,AFG,Afghanistan,Production,tonnes,"27,694"
3,ZAF,Afrique du Sud,tonne_co2/tonne_poulet,kg CO2eq/kg,0
4,ZAF,Afrique du Sud,emission_co2(tonnes),kilotonnes,572


### 3.6.2 - Pivotement du df pour passer les valeurs de la série "element" en colonnes

In [83]:
GES_poulet = GES_poulet.pivot(index="pays",columns="element",values="valeur").reset_index()
GES_poulet.head()

element,pays,Production,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,"27,694",10,0
1,Afrique du Sud,"1,658,159",572,0
2,Albanie,"12,912",10,1
3,Algérie,"263,976",462,2
4,Allemagne,"1,009,565",416,0


###  3.6.3 - Suppression de la colonne "production"

In [84]:
GES_poulet = GES_poulet.drop("Production",axis=1)
GES_poulet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pays                    200 non-null    object 
 1   emission_co2(tonnes)    200 non-null    float64
 2   tonne_co2/tonne_poulet  200 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.8+ KB


### 3.6.4 - Renommage des colonnes

In [85]:
GES_poulet["emission_co2(tonnes)"] = GES_poulet["emission_co2(tonnes)"]*1000
GES_poulet["tonne_co2/tonne_poulet"] = GES_poulet["tonne_co2/tonne_poulet"]*1000
GES_poulet.head()

element,pays,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,"10,058",363
1,Afrique du Sud,"571,841",345
2,Albanie,"10,486",812
3,Algérie,"461,882","1,750"
4,Allemagne,"415,700",412


## 3.7 - Création du merge "produit_poulet" par le merge des df "produit_poulet" et "GES_poulet"

In [86]:
produit_poulet = pd.merge(produit_poulet,GES_poulet, on="pays",how="outer")
produit_poulet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           203 non-null    object 
 1   code_pays                      178 non-null    object 
 2   production                     178 non-null    float64
 3   cout_production(USD/t)         42 non-null     float64
 4   import_tonnes                  178 non-null    float64
 5   import_USD                     178 non-null    float64
 6   prix_import_USD/tonne          178 non-null    float64
 7   importFrance_tonnes            95 non-null     float64
 8   importFrance_USD               95 non-null     float64
 9   prix_importFrance_USD/tonne    100 non-null    float64
 10  export_tonnes                  178 non-null    float64
 11  export_USD                     178 non-null    float64
 12  prix_export_USD/tonne          178 non-null    flo

In [87]:
code_null = produit_poulet[produit_poulet["code_pays"].isnull()]
code_pays_null = code_null["pays"].tolist()
print(code_pays_null)

['Bahreïn', 'Pays-Bas', 'Qatar', 'Singapour', 'Bhoutan', 'Brunéi Darussalam', 'Guadeloupe', 'Guinée équatoriale', 'Guyane française', 'Madagascar', 'Martinique', 'Micronésie (États fédérés de)', 'Nauru', 'Nioué', 'Palestine', 'Porto Rico', 'Réunion', 'Somalie', 'Soudan du Sud', 'Tokélaou', 'Tonga', 'Tuvalu', 'Équateur', 'Érythrée', 'Îles Cook']


In [88]:
produit_poulet.head()

,pays,code_pays,production,cout_production(USD/t),import_tonnes,import_USD,prix_import_USD/tonne,importFrance_tonnes,importFrance_USD,prix_importFrance_USD/tonne,...,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,AFG,"27,850",NaN,"28,937","28,934,000","1,000",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"-6,727","64,000","63,514","10,058",363
1,Afrique du Sud,ZAF,"1,658,159","2,017","480,346","410,502,000",855,0,"2,000","25,000",...,NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000","571,841",345
2,Albanie,ALB,"12,912",NaN,"17,515","16,481,000",941,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,"33,000","30,192","10,486",812
3,Algérie,DZA,"262,965","2,181",0,"2,000","5,000",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,"286,000","262,965","461,882","1,750"
4,Allemagne,DEU,"1,009,565","1,460","474,951","1,101,592,000","2,319","19,245","69,354,000","3,604",...,"32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829","415,700",412


# 4 - Regroupement pays
## 4.1 - df delai_douanes
### 4.1.1 - Lecture df

In [89]:
delai_douanes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   country        217 non-null    object 
 1   code_pays      217 non-null    object 
 2   duree(heures)  187 non-null    float64
dtypes: float64(1), object(2)
memory usage: 5.2+ KB


In [90]:
delai_douanes.head()

,country,code_pays,duree(heures)
0,Afghanistan,AFG,96
1,Albania,ALB,10
2,Algeria,DZA,327
3,American Samoa,ASM,NaN
4,Andorra,AND,NaN


## 4.2 - df cout_douanes
### 4.2.1 - Lecture df

In [91]:
cout_douanes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            217 non-null    object 
 1   code_pays          217 non-null    object 
 2   cout_douanes(USD)  187 non-null    float64
dtypes: float64(1), object(2)
memory usage: 5.2+ KB


In [92]:
cout_douanes.head()

,country,code_pays,cout_douanes(USD)
0,Afghanistan,AFG,750
1,Albania,ALB,77
2,Algeria,DZA,409
3,American Samoa,ASM,NaN
4,Andorra,AND,NaN


### 4.2.2 - Création du df "douanes" avec le merge des df "cout_douanes" et "delai_douanes"

In [93]:
douanes = pd.merge(cout_douanes, delai_douanes, on="code_pays", how="inner")
douanes = douanes[["country_x","code_pays","cout_douanes(USD)","duree(heures)"]]
douanes.rename(columns={"country_x" : "country","duree(heures)":"delai_douanes(heures)"},inplace=True)
douanes.head()

,country,code_pays,cout_douanes(USD),delai_douanes(heures)
0,Afghanistan,AFG,750,96
1,Albania,ALB,77,10
2,Algeria,DZA,409,327
3,American Samoa,ASM,NaN,NaN
4,Andorra,AND,NaN,NaN


#### Les noms des pays est au format de langue anglaise. Je les passerai en français lors du prochain merge, grâce au code_pays (iso3)
## 4.3 - df stabilite
### 4.3.1 - lecture df

In [94]:
stabilite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_pays                   197 non-null    object 
 1   pays                        197 non-null    object 
 2   indice_stabilite_politique  197 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.7+ KB


In [95]:
stabilite.head()

,code_pays,pays,indice_stabilite_politique
0,NLD,Pays-Bas (Royaume des),1
1,AFG,Afghanistan,-3
2,ZAF,Afrique du Sud,-0
3,ALB,Albanie,0
4,DZA,Algérie,-1


### 4.3.2 - Renommage de la colonne "indice_stabilite_politique" en "stabilite"

In [96]:
stabilite.rename(columns={"indice_stabilite_politique":"stabilite"},inplace=True)

### 4.3.3 - mise sous format à 2 décimales des valeurs de la colonne "stabilite"

In [97]:
stabilite["stabilite"] = stabilite["stabilite"].apply(lambda x: "{:.2f}".format(x))#.astype(float)
stabilite.head()

,code_pays,pays,stabilite
0,NLD,Pays-Bas (Royaume des),0.92
1,AFG,Afghanistan,-2.80
2,ZAF,Afrique du Sud,-0.28
3,ALB,Albanie,0.38
4,DZA,Algérie,-0.92


In [98]:
stabilite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   code_pays  197 non-null    object
 1   pays       197 non-null    object
 2   stabilite  197 non-null    object
dtypes: object(3)
memory usage: 4.7+ KB


### 4.3.4 - C'éation du df "pays" avec le merge des df "douanes" et "stabilite"

In [99]:
pays = pd.merge(douanes, stabilite, on="code_pays", how="left")
pays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                217 non-null    object 
 1   code_pays              217 non-null    object 
 2   cout_douanes(USD)      187 non-null    float64
 3   delai_douanes(heures)  187 non-null    float64
 4   pays                   196 non-null    object 
 5   stabilite              196 non-null    object 
dtypes: float64(2), object(4)
memory usage: 10.3+ KB


In [100]:
pays.head()

,country,code_pays,cout_douanes(USD),delai_douanes(heures),pays,stabilite
0,Afghanistan,AFG,750,96,Afghanistan,-2.80
1,Albania,ALB,77,10,Albanie,0.38
2,Algeria,DZA,409,327,Algérie,-0.92
3,American Samoa,ASM,NaN,NaN,Samoa américaines,1.22
4,Andorra,AND,NaN,NaN,Andorre,1.42


### 4.3.5 - Choix des colonnes à conserver et modification de leur ordre d'affichage

In [101]:
pays = pays[["pays","code_pays","cout_douanes(USD)","delai_douanes(heures)","stabilite"]]
pays.head()

,pays,code_pays,cout_douanes(USD),delai_douanes(heures),stabilite
0,Afghanistan,AFG,750,96,-2.80
1,Albanie,ALB,77,10,0.38
2,Algérie,DZA,409,327,-0.92
3,Samoa américaines,ASM,NaN,NaN,1.22
4,Andorre,AND,NaN,NaN,1.42


## 4.4 - df geo
### 4.4.1 - Lecture df

In [102]:
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   iso3       238 non-null    object 
 1   country    238 non-null    object 
 2   pays       238 non-null    object 
 3   continent  238 non-null    object 
 4   lat        238 non-null    float64
 5   lon        238 non-null    float64
dtypes: float64(2), object(4)
memory usage: 11.3+ KB


In [103]:
geo.head()

,iso3,country,pays,continent,lat,lon
0,ABW,Aruba,Aruba,America,13,-70
1,AFG,Afghanistan,Afghanistan,Asia,35,69
2,AGO,Angola,Angola,Africa,-9,13
3,AIA,Anguilla,Anguilla,America,18,-63
4,ALB,Albania,Albanie,Europe,41,20


### 4.4.2 - renommage de la colonne "iso3" en "code_pays"

In [104]:
geo.rename(columns={"iso3":"code_pays"},inplace=True)

### 4.4.3 - Merge des df "geo" et "pays"

In [105]:
pays = pd.merge(pays,geo, on="code_pays",how="inner")
pays.head()

,pays_x,code_pays,cout_douanes(USD),delai_douanes(heures),stabilite,country,pays_y,continent,lat,lon
0,Afghanistan,AFG,750,96,-2.80,Afghanistan,Afghanistan,Asia,35,69
1,Albanie,ALB,77,10,0.38,Albania,Albanie,Europe,41,20
2,Algérie,DZA,409,327,-0.92,Algeria,Algérie,Africa,37,3
3,Andorre,AND,NaN,NaN,1.42,Andorra,Andorre,Europe,42,2
4,Angola,AGO,"1,030",96,-0.38,Angola,Angola,Africa,-9,13


### 4.4.6 - Choix des colonnes à conserver et modification de leur ordre d'affichage

In [106]:
pays = pays[["pays_y","code_pays","cout_douanes(USD)","delai_douanes(heures)","stabilite","lat","lon"]]
pays.rename(columns={"pays_y" : "pays"},inplace=True)
pays.head()

,pays,code_pays,cout_douanes(USD),delai_douanes(heures),stabilite,lat,lon
0,Afghanistan,AFG,750,96,-2.80,35,69
1,Albanie,ALB,77,10,0.38,41,20
2,Algérie,DZA,409,327,-0.92,37,3
3,Andorre,AND,NaN,NaN,1.42,42,2
4,Angola,AGO,"1,030",96,-0.38,-9,13


### 4.4.7 - Affichage des valeurs de "lat" et "lon" à 5 décimales

In [107]:
pays["lat"] = pays["lat"].map(lambda x: "{:.5f}".format(x))
pays["lon"] = pays["lon"].map(lambda x: "{:.5f}".format(x))
pays.head()

,pays,code_pays,cout_douanes(USD),delai_douanes(heures),stabilite,lat,lon
0,Afghanistan,AFG,750,96,-2.80,34.51667,69.20000
1,Albanie,ALB,77,10,0.38,41.33333,19.81667
2,Algérie,DZA,409,327,-0.92,36.83333,3.00000
3,Andorre,AND,NaN,NaN,1.42,42.50000,1.50000
4,Angola,AGO,"1,030",96,-0.38,-8.83333,13.25000


# 5 - Affinage
## 5.1 - Regroupement des df principaux en un seul "df_final"
### 5.1.1 - 1er merge ( "population" et "pays")

In [108]:
df_final = pd.merge(population,pays,on="pays", how="outer")

### 5.1.2 - choix de l'ordre des colonnes

In [109]:
df_final = df_final[["pays","code_pays_x","population","%_evolution_2007/2017","cout_viande_pers/an(USD)","PIB_habitant(USD)",
                    "Tx_inflation","dispo_alimentaire(kg/pers/an)","cout_douanes(USD)","delai_douanes(heures)","stabilite","lat","lon"]]
df_final.rename(columns={"code_pays_x" : "code_pays"},inplace=True)

In [110]:
df_final.head()

,pays,code_pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD),Tx_inflation,dispo_alimentaire(kg/pers/an),cout_douanes(USD),delai_douanes(heures),stabilite,lat,lon
0,Afghanistan,AFG,"35,643,418",38,0,"2,058",7,2,750,96,-2.80,34.51667,69.20000
1,Afrique du Sud,ZAF,"56,641,209",13,387,"13,860",7,36,668,87,-0.28,-25.73000,28.22000
2,Afrique du Sud,ZAF,"56,641,209",13,387,"13,860",7,36,668,87,-0.28,-33.93333,18.46667
3,Albanie,ALB,"2,879,355",-3,439,"12,771",4,11,77,10,0.38,41.33333,19.81667
4,Algérie,DZA,"41,136,546",21,352,"11,737",5,6,409,327,-0.92,36.83333,3.00000


In [111]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           285 non-null    object 
 1   code_pays                      244 non-null    object 
 2   population                     245 non-null    float64
 3   %_evolution_2007/2017          245 non-null    float64
 4   cout_viande_pers/an(USD)       245 non-null    float64
 5   PIB_habitant(USD)              245 non-null    float64
 6   Tx_inflation                   245 non-null    float64
 7   dispo_alimentaire(kg/pers/an)  245 non-null    float64
 8   cout_douanes(USD)              192 non-null    float64
 9   delai_douanes(heures)          192 non-null    float64
 10  stabilite                      201 non-null    object 
 11  lat                            212 non-null    object 
 12  lon                            212 non-null    obj

### 5.1.3 - 2ème merge ( "df_final" et "produit_poulet")

In [112]:
df_final = pd.merge(df_final,produit_poulet, on="pays", how="outer")

### 5.1.4 - choix de l'ordre des colonnes

In [113]:
df_final = df_final[["pays","code_pays_x","population","%_evolution_2007/2017","cout_viande_pers/an(USD)","PIB_habitant(USD)",
                    "Tx_inflation","dispo_alimentaire(kg/pers/an)","cout_douanes(USD)","delai_douanes(heures)","stabilite","lat","lon","production",
                     "cout_production(USD/t)","import_tonnes","import_USD","prix_import_USD/tonne","importFrance_tonnes",
                     "importFrance_USD","prix_importFrance_USD/tonne","export_tonnes","export_USD","prix_export_USD/tonne",
                     "exportFrance_tonnes","exportFrance_USD","prix_exportFrance_USD/tonne","excedent_commercial_fr_tonnes",
                     "excedent_commercial_fr_USD","variation_stock","nourriture","disponibilite_interieure","emission_co2(tonnes)",
                     "tonne_co2/tonne_poulet"]]
df_final.rename(columns={"code_pays_x" : "code_pays"},inplace=True)

In [114]:
df_final.head()

,pays,code_pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD),Tx_inflation,dispo_alimentaire(kg/pers/an),cout_douanes(USD),delai_douanes(heures),...,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,AFG,"35,643,418",38,0,"2,058",7,2,750,96,...,NaN,NaN,NaN,NaN,NaN,"-6,727","64,000","63,514","10,058",363
1,Afrique du Sud,ZAF,"56,641,209",13,387,"13,860",7,36,668,87,...,NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000","571,841",345
2,Afrique du Sud,ZAF,"56,641,209",13,387,"13,860",7,36,668,87,...,NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000","571,841",345
3,Albanie,ALB,"2,879,355",-3,439,"12,771",4,11,77,10,...,NaN,NaN,NaN,NaN,NaN,0,"33,000","30,192","10,486",812
4,Algérie,DZA,"41,136,546",21,352,"11,737",5,6,409,327,...,NaN,NaN,NaN,NaN,NaN,0,"286,000","262,965","461,882","1,750"


In [115]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           285 non-null    object 
 1   code_pays                      244 non-null    object 
 2   population                     245 non-null    float64
 3   %_evolution_2007/2017          245 non-null    float64
 4   cout_viande_pers/an(USD)       245 non-null    float64
 5   PIB_habitant(USD)              245 non-null    float64
 6   Tx_inflation                   245 non-null    float64
 7   dispo_alimentaire(kg/pers/an)  245 non-null    float64
 8   cout_douanes(USD)              192 non-null    float64
 9   delai_douanes(heures)          192 non-null    float64
 10  stabilite                      201 non-null    object 
 11  lat                            212 non-null    object 
 12  lon                            212 non-null    obj

#### 5.1.4.2 - Suppression des doublons 

In [116]:
df_final = df_final[~df_final["pays"].duplicated(keep="first")]

### 5.1.5 - exportation de "df_final" au format .xlsx et nettoyage de quelques données ayant des différences dans leurs appélations, sous excel 

In [117]:
#df_final.to_excel("df_final.xlsx")

### 5.1.6 - Réimportation du df_final après nettoyage sous excel, nommé "df_final2", cette fois-ci

In [118]:
df_final2 = pd.read_excel("df_final.xlsx")

In [119]:
df_final2.head()

,pays,code_pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD),Tx_inflation,dispo_alimentaire(kg/pers/an),cout_douanes(USD),delai_douanes(heures),...,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,AFG,35643418,38,0,"2,058",7,2,750,96,...,NaN,NaN,NaN,NaN,NaN,"-6,727","64,000","63,514","10,058",363
1,Afrique du Sud,ZAF,56641209,13,387,"13,860",7,36,668,87,...,NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000","571,841",345
2,Albanie,ALB,2879355,-3,439,"12,771",4,11,77,10,...,NaN,NaN,NaN,NaN,NaN,0,"33,000","30,192","10,486",812
3,Algerie,DZA,41136546,21,352,"11,737",5,6,409,327,...,NaN,NaN,NaN,NaN,NaN,0,"286,000","262,965","461,882","1,750"
4,Allemagne,DEU,82624374,2,243,"53,072",3,11,0,0,...,"32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829","415,700",412


In [120]:
df_final2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           235 non-null    object 
 1   code_pays                      235 non-null    object 
 2   population                     235 non-null    int64  
 3   %_evolution_2007/2017          235 non-null    float64
 4   cout_viande_pers/an(USD)       235 non-null    float64
 5   PIB_habitant(USD)              235 non-null    float64
 6   Tx_inflation                   235 non-null    float64
 7   dispo_alimentaire(kg/pers/an)  235 non-null    float64
 8   cout_douanes(USD)              179 non-null    float64
 9   delai_douanes(heures)          179 non-null    float64
 10  stabilite                      188 non-null    float64
 11  lat                            199 non-null    float64
 12  lon                            199 non-null    flo

### 5.1.7 - Sélection des pays ayant des valeurs nulles dans les 25 colonnes suivantes :

In [121]:
colnull1 =["cout_douanes(USD)","delai_douanes(heures)","stabilite","lat","lon","production","cout_production(USD/t)",
         "import_tonnes","import_USD","prix_import_USD/tonne","importFrance_tonnes","importFrance_USD",
         "prix_importFrance_USD/tonne","export_tonnes","export_USD","prix_export_USD/tonne","exportFrance_tonnes",
         "exportFrance_USD","prix_exportFrance_USD/tonne","excedent_commercial_fr_tonnes","excedent_commercial_fr_USD",
         "variation_stock","nourriture","disponibilite_interieure","emission_co2(tonnes)","tonne_co2/tonne_poulet"]

In [122]:
df_null1 = df_final2[colnull1].isnull()

In [123]:
lidrop1  = df_final2[df_null1.all(axis=1)]

In [124]:
print(lidrop1)

                                             pays code_pays  population  \
7                                        Anguilla       AIA       15015   
27                Bonaire, Saint-Eustache et Saba       BES       24366   
53                                        Curaçao       CUW      171524   
80                                           Guam       GUM      168606   
90                                     Ile de Man       IMN       83580   
93                       Iles Falkland (Malvinas)       FLK        3598   
99                       Iles Vierges americaines       VIR      101851   
101                         Iles Wallis-et-Futuna       WLF       11936   
123                                 Liechtenstein       LIE       37889   
137                                       Mayotte       MYT      271565   
140                                        Monaco       MCO       37044   
143                                    Montserrat       MSR        4699   
186                      

In [125]:
nombre_lignes_concernees = lidrop1.shape[0]
print("Nombre de lignes concernées :", nombre_lignes_concernees)

Nombre de lignes concernées : 20


### 5.1.8 - Suppression des pays concernés

In [126]:
df_final2= df_final2.drop(lidrop1.index)

In [127]:
df_final2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215 entries, 0 to 234
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           215 non-null    object 
 1   code_pays                      215 non-null    object 
 2   population                     215 non-null    int64  
 3   %_evolution_2007/2017          215 non-null    float64
 4   cout_viande_pers/an(USD)       215 non-null    float64
 5   PIB_habitant(USD)              215 non-null    float64
 6   Tx_inflation                   215 non-null    float64
 7   dispo_alimentaire(kg/pers/an)  215 non-null    float64
 8   cout_douanes(USD)              179 non-null    float64
 9   delai_douanes(heures)          179 non-null    float64
 10  stabilite                      188 non-null    float64
 11  lat                            199 non-null    float64
 12  lon                            199 non-null    float64


In [128]:
df_final2.head()

,pays,code_pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD),Tx_inflation,dispo_alimentaire(kg/pers/an),cout_douanes(USD),delai_douanes(heures),...,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,AFG,35643418,38,0,"2,058",7,2,750,96,...,NaN,NaN,NaN,NaN,NaN,"-6,727","64,000","63,514","10,058",363
1,Afrique du Sud,ZAF,56641209,13,387,"13,860",7,36,668,87,...,NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000","571,841",345
2,Albanie,ALB,2879355,-3,439,"12,771",4,11,77,10,...,NaN,NaN,NaN,NaN,NaN,0,"33,000","30,192","10,486",812
3,Algerie,DZA,41136546,21,352,"11,737",5,6,409,327,...,NaN,NaN,NaN,NaN,NaN,0,"286,000","262,965","461,882","1,750"
4,Allemagne,DEU,82624374,2,243,"53,072",3,11,0,0,...,"32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829","415,700",412


### 5.1.9 - Suppression de la Chine (entant que cumul des 4 régions la composant)

In [129]:
drop_chine = df_final2.loc[df_final2["pays"]=="Chine"]
df_final2 = df_final2.drop(drop_chine.index)

In [130]:
df_final2.head()

,pays,code_pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD),Tx_inflation,dispo_alimentaire(kg/pers/an),cout_douanes(USD),delai_douanes(heures),...,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,AFG,35643418,38,0,"2,058",7,2,750,96,...,NaN,NaN,NaN,NaN,NaN,"-6,727","64,000","63,514","10,058",363
1,Afrique du Sud,ZAF,56641209,13,387,"13,860",7,36,668,87,...,NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000","571,841",345
2,Albanie,ALB,2879355,-3,439,"12,771",4,11,77,10,...,NaN,NaN,NaN,NaN,NaN,0,"33,000","30,192","10,486",812
3,Algerie,DZA,41136546,21,352,"11,737",5,6,409,327,...,NaN,NaN,NaN,NaN,NaN,0,"286,000","262,965","461,882","1,750"
4,Allemagne,DEU,82624374,2,243,"53,072",3,11,0,0,...,"32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829","415,700",412


In [131]:
df_final2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 214 entries, 0 to 234
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           214 non-null    object 
 1   code_pays                      214 non-null    object 
 2   population                     214 non-null    int64  
 3   %_evolution_2007/2017          214 non-null    float64
 4   cout_viande_pers/an(USD)       214 non-null    float64
 5   PIB_habitant(USD)              214 non-null    float64
 6   Tx_inflation                   214 non-null    float64
 7   dispo_alimentaire(kg/pers/an)  214 non-null    float64
 8   cout_douanes(USD)              178 non-null    float64
 9   delai_douanes(heures)          178 non-null    float64
 10  stabilite                      188 non-null    float64
 11  lat                            198 non-null    float64
 12  lon                            198 non-null    float64


## 5.2 - Nouvelle répartition de "df_final2" en 3 df distincts
### 5.2.1 - population

In [132]:
population = df_final2[["pays","population","%_evolution_2007/2017","cout_viande_pers/an(USD)","PIB_habitant(USD)",
                        "dispo_alimentaire(kg/pers/an)","Tx_inflation"]]
population.info()

<class 'pandas.core.frame.DataFrame'>
Index: 214 entries, 0 to 234
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           214 non-null    object 
 1   population                     214 non-null    int64  
 2   %_evolution_2007/2017          214 non-null    float64
 3   cout_viande_pers/an(USD)       214 non-null    float64
 4   PIB_habitant(USD)              214 non-null    float64
 5   dispo_alimentaire(kg/pers/an)  214 non-null    float64
 6   Tx_inflation                   214 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 13.4+ KB


In [133]:
pays_iso = df_final2[["pays", "code_pays"]]
pays_iso.info()

<class 'pandas.core.frame.DataFrame'>
Index: 214 entries, 0 to 234
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pays       214 non-null    object
 1   code_pays  214 non-null    object
dtypes: object(2)
memory usage: 5.0+ KB


In [134]:
#pays_iso.to_excel("pays_iso.xlsx")

### 5.2.2 - pays

In [135]:
pays = df_final2[["pays","cout_douanes(USD)","delai_douanes(heures)","stabilite","lat","lon"]]
pays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 214 entries, 0 to 234
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pays                   214 non-null    object 
 1   cout_douanes(USD)      178 non-null    float64
 2   delai_douanes(heures)  178 non-null    float64
 3   stabilite              188 non-null    float64
 4   lat                    198 non-null    float64
 5   lon                    198 non-null    float64
dtypes: float64(5), object(1)
memory usage: 11.7+ KB


### 5.2.3 - produit_poulet

In [136]:
produit_poulet = df_final2 [["pays","production","cout_production(USD/t)","import_tonnes","import_USD","prix_import_USD/tonne",
                             "importFrance_tonnes","importFrance_USD","prix_importFrance_USD/tonne","export_tonnes","export_USD",
                             "prix_export_USD/tonne","exportFrance_tonnes","exportFrance_USD","prix_exportFrance_USD/tonne",
                             "excedent_commercial_fr_tonnes","excedent_commercial_fr_USD","variation_stock","nourriture",
                             "disponibilite_interieure","emission_co2(tonnes)","tonne_co2/tonne_poulet"]]
produit_poulet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 214 entries, 0 to 234
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           214 non-null    object 
 1   production                     177 non-null    float64
 2   cout_production(USD/t)         42 non-null     float64
 3   import_tonnes                  177 non-null    float64
 4   import_USD                     177 non-null    float64
 5   prix_import_USD/tonne          177 non-null    float64
 6   importFrance_tonnes            94 non-null     float64
 7   importFrance_USD               94 non-null     float64
 8   prix_importFrance_USD/tonne    99 non-null     float64
 9   export_tonnes                  177 non-null    float64
 10  export_USD                     177 non-null    float64
 11  prix_export_USD/tonne          177 non-null    float64
 12  exportFrance_tonnes            32 non-null     float64


In [137]:
produit_poulet.head()

,pays,production,cout_production(USD/t),import_tonnes,import_USD,prix_import_USD/tonne,importFrance_tonnes,importFrance_USD,prix_importFrance_USD/tonne,export_tonnes,...,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,"27,850",NaN,"28,937","28,934,000","1,000",NaN,NaN,NaN,37,...,NaN,NaN,NaN,NaN,NaN,"-6,727","64,000","63,514","10,058",363
1,Afrique du Sud,"1,658,159","2,017","480,346","410,502,000",855,0,"2,000","25,000","58,259",...,NaN,NaN,0,NaN,NaN,"5,700","2,110,000","2,070,000","571,841",345
2,Albanie,"12,912",NaN,"17,515","16,481,000",941,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,0,"33,000","30,192","10,486",812
3,Algerie,"262,965","2,181",0,"2,000","5,000",NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,0,"286,000","262,965","461,882","1,750"
4,Allemagne,"1,009,565","1,460","474,951","1,101,592,000","2,319","19,245","69,354,000","3,604","300,561",...,"32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829","415,700",412


## 5.3 - Nouveau nettoyage approfondi
### 5.3.1 - Sélection des pays n'ayant aucune donnée de production, importation, exportation, disponibilité, etc.

In [138]:
colprodnull =["production","cout_production(USD/t)","import_tonnes","import_USD","prix_import_USD/tonne","importFrance_tonnes",
              "importFrance_USD","prix_importFrance_USD/tonne","export_tonnes","export_USD","prix_export_USD/tonne",
              "exportFrance_tonnes","exportFrance_USD","prix_exportFrance_USD/tonne","excedent_commercial_fr_tonnes",
              "excedent_commercial_fr_USD","variation_stock","nourriture","disponibilite_interieure","emission_co2(tonnes)",
              "tonne_co2/tonne_poulet"]

df_prodnull = produit_poulet[colprodnull].isnull()

prod_drop  = produit_poulet[df_prodnull.all(axis=1)]

In [139]:
print(prod_drop)

                          pays  production  cout_production(USD/t)  \
5                      Andorre         NaN                     NaN   
12                       Aruba         NaN                     NaN   
24                    Bermudes         NaN                     NaN   
75                   Gibraltar         NaN                     NaN   
78                   Groenland         NaN                     NaN   
91                iles Caïmans         NaN                     NaN   
94                  Iles Feroe         NaN                     NaN   
95      Iles Mariannes du Nord         NaN                     NaN   
96               Iles Marshall         NaN                     NaN   
98     Iles Turques-et-Caïques         NaN                     NaN   
100  Iles Vierges britanniques         NaN                     NaN   
160                     Palaos         NaN                     NaN   
191                Saint-Marin         NaN                     NaN   

     import_tonnes 

In [140]:
nombre_lignes_prod_drop = prod_drop.shape[0]
print("Nombre de lignes concernées :", nombre_lignes_prod_drop)

Nombre de lignes concernées : 13


### 5.3.2 - Crétion de la liste "pays_drop" contenant les pays n'ayant aucune des valeurs sélectionnées

In [141]:
liste_pays_drop = prod_drop["pays"]

In [142]:
pays_drop = liste_pays_drop.tolist()

In [143]:
print(pays_drop)

['Andorre', 'Aruba', 'Bermudes', 'Gibraltar', 'Groenland', 'iles Caïmans', 'Iles Feroe', 'Iles Mariannes du Nord', 'Iles Marshall', 'Iles Turques-et-Caïques', 'Iles Vierges britanniques', 'Palaos', 'Saint-Marin']


### 5.3.3 - Suppression des pays dans le df "produit_poulet"

In [144]:
produit_poulet= produit_poulet.drop(prod_drop.index)

In [145]:
produit_poulet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 0 to 234
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           201 non-null    object 
 1   production                     177 non-null    float64
 2   cout_production(USD/t)         42 non-null     float64
 3   import_tonnes                  177 non-null    float64
 4   import_USD                     177 non-null    float64
 5   prix_import_USD/tonne          177 non-null    float64
 6   importFrance_tonnes            94 non-null     float64
 7   importFrance_USD               94 non-null     float64
 8   prix_importFrance_USD/tonne    99 non-null     float64
 9   export_tonnes                  177 non-null    float64
 10  export_USD                     177 non-null    float64
 11  prix_export_USD/tonne          177 non-null    float64
 12  exportFrance_tonnes            32 non-null     float64


### 5.3.4 - Suprression des pays dans le df "population"

In [146]:
population= population.drop(prod_drop.index)

In [147]:
population.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 0 to 234
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           201 non-null    object 
 1   population                     201 non-null    int64  
 2   %_evolution_2007/2017          201 non-null    float64
 3   cout_viande_pers/an(USD)       201 non-null    float64
 4   PIB_habitant(USD)              201 non-null    float64
 5   dispo_alimentaire(kg/pers/an)  201 non-null    float64
 6   Tx_inflation                   201 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 12.6+ KB


In [148]:
population.head()

,pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD),dispo_alimentaire(kg/pers/an),Tx_inflation
0,Afghanistan,35643418,38,0,"2,058",2,7
1,Afrique du Sud,56641209,13,387,"13,860",36,7
2,Albanie,2879355,-3,439,"12,771",11,4
3,Algerie,41136546,21,352,"11,737",6,5
4,Allemagne,82624374,2,243,"53,072",11,3


### 5.3.5 - Suppression des pays dans le df "pays"

In [149]:
pays = pays[~pays["pays"].isin(pays_drop)]

In [150]:
pays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 0 to 234
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pays                   201 non-null    object 
 1   cout_douanes(USD)      175 non-null    float64
 2   delai_douanes(heures)  175 non-null    float64
 3   stabilite              183 non-null    float64
 4   lat                    185 non-null    float64
 5   lon                    185 non-null    float64
dtypes: float64(5), object(1)
memory usage: 11.0+ KB


### 5.3.6 - Vérification d'absence de valeurs nulles dans les coordonnées

In [151]:
missed_coords = pays[pd.isna(pays["lat"]) | pd.isna(pays["lon"])]
missed_coords_list = missed_coords["pays"].tolist()
print(missed_coords_list)

['Chine, continentale', 'Chine, Taiwan Province de', 'Guadeloupe', 'Guyane française', 'Iles Cook', 'Martinique', 'Montenegro', 'Nioue', 'Palestine', 'Republique democratique du Congo', 'Reunion', 'Roumanie', 'Serbie', 'Soudan du Sud', 'Timor-Leste', 'Tokelaou']


### 5.3.7 - Création d'une liste établissant les coordonnées manquantes à insérer dans le df "pays"

In [152]:
from geopy.geocoders import Nominatim

In [153]:
def get_coordinates(country):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(country)
    
    if location:
        lat = round(location.latitude, 5)
        lon = round(location.longitude, 5)
        return lat, lon
    else:
        return None, None

def get_all_coordinates(country_list):
    coord_dict = {}
    
    for country in country_list:
        lat, lon = get_coordinates(country)
        coord_dict[country] = (lat, lon)
    
    return coord_dict

coord_dict = get_all_coordinates(missed_coords_list)

for country, coords in coord_dict.items():
    if coords[0] is not None and coords[1] is not None:
        print(f"Les coordonnées de {country} sont : {coords[0]}, {coords[1]}")
    else:
        print(f"Impossible de trouver les coordonnées de {country}")

Impossible de trouver les coordonnées de Chine, continentale
Les coordonnées de Chine, Taiwan Province de sont : 23.87119, 120.92341
Les coordonnées de Guadeloupe sont : 16.25288, -61.56869
Les coordonnées de Guyane française sont : 4.00399, -53.0
Les coordonnées de Iles Cook sont : -19.99697, -157.78587
Les coordonnées de Martinique sont : 14.61137, -60.96208
Les coordonnées de Montenegro sont : 42.98689, 19.5181
Les coordonnées de Nioue sont : -19.05364, -169.86134
Les coordonnées de Palestine sont : 31.46242, 34.26272
Les coordonnées de Republique democratique du Congo sont : -2.98143, 23.82226
Les coordonnées de Reunion sont : -21.13074, 55.53648
Les coordonnées de Roumanie sont : 45.98521, 24.68592
Les coordonnées de Serbie sont : 44.15341, 20.55144
Les coordonnées de Soudan du Sud sont : 7.86994, 29.66679
Les coordonnées de Timor-Leste sont : -8.74432, 126.06348
Les coordonnées de Tokelaou sont : -9.16764, -171.81969


In [154]:
print(coord_dict)

{'Chine, continentale': (None, None), 'Chine, Taiwan Province de': (23.87119, 120.92341), 'Guadeloupe': (16.25288, -61.56869), 'Guyane française': (4.00399, -53.0), 'Iles Cook': (-19.99697, -157.78587), 'Martinique': (14.61137, -60.96208), 'Montenegro': (42.98689, 19.5181), 'Nioue': (-19.05364, -169.86134), 'Palestine': (31.46242, 34.26272), 'Republique democratique du Congo': (-2.98143, 23.82226), 'Reunion': (-21.13074, 55.53648), 'Roumanie': (45.98521, 24.68592), 'Serbie': (44.15341, 20.55144), 'Soudan du Sud': (7.86994, 29.66679), 'Timor-Leste': (-8.74432, 126.06348), 'Tokelaou': (-9.16764, -171.81969)}


### 5.3.8 - Création d'une boucle pour l'insertion des valeurs manquantes dans les colonnes "lat" et "lon" à partir de la liste "coord_dict".

In [155]:
for country, coords in coord_dict.items():
    lat,lon = coords
    pays.loc[pays["pays"] == country, "lat"] = lat
    pays.loc[pays["pays"] == country, "lon"] = lon

#### 5.3.8.1 - Remplacement manuel des valeurs nulles concernant la chine continentale, pour "lat" et "lon"

In [156]:
pays.loc[pays["pays"] == "Chine, continentale", "lat"] = 39.90571
pays.loc[pays["pays"] == "Chine, continentale", "lon"] = 116.39129

In [157]:
pays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 0 to 234
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pays                   201 non-null    object 
 1   cout_douanes(USD)      175 non-null    float64
 2   delai_douanes(heures)  175 non-null    float64
 3   stabilite              183 non-null    float64
 4   lat                    201 non-null    float64
 5   lon                    201 non-null    float64
dtypes: float64(5), object(1)
memory usage: 11.0+ KB


### 5.3.9 - passage des valeurs de "stabilite" à 2 décimales et "lat" , "lon" à 5 décimales.

In [158]:
pays["stabilite"] = pays["stabilite"].map(lambda x: "{:.2f}".format(x))
# pays["lat"] = pays["lat"].map(lambda x: "{:.5f}".format(x))
# pays["lon"] = pays["lon"].map(lambda x: "{:.5f}".format(x))
pays.head()

,pays,cout_douanes(USD),delai_douanes(heures),stabilite,lat,lon
0,Afghanistan,750,96,-2.80,35,69
1,Afrique du Sud,668,87,-0.28,-26,28
2,Albanie,77,10,0.38,41,20
3,Algerie,409,327,-0.92,37,3
4,Allemagne,0,0,0.59,53,13


In [159]:
pays.head()

,pays,cout_douanes(USD),delai_douanes(heures),stabilite,lat,lon
0,Afghanistan,750,96,-2.80,35,69
1,Afrique du Sud,668,87,-0.28,-26,28
2,Albanie,77,10,0.38,41,20
3,Algerie,409,327,-0.92,37,3
4,Allemagne,0,0,0.59,53,13


## 5.4 - Remplacement des valeurs nulles par la valeur "0.0"
### 5.4.1 du df "produit_poulet" 

In [160]:
produit_poulet = produit_poulet.fillna(0.0)
produit_poulet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 0 to 234
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           201 non-null    object 
 1   production                     201 non-null    float64
 2   cout_production(USD/t)         201 non-null    float64
 3   import_tonnes                  201 non-null    float64
 4   import_USD                     201 non-null    float64
 5   prix_import_USD/tonne          201 non-null    float64
 6   importFrance_tonnes            201 non-null    float64
 7   importFrance_USD               201 non-null    float64
 8   prix_importFrance_USD/tonne    201 non-null    float64
 9   export_tonnes                  201 non-null    float64
 10  export_USD                     201 non-null    float64
 11  prix_export_USD/tonne          201 non-null    float64
 12  exportFrance_tonnes            201 non-null    float64


In [161]:
produit_poulet.head()

,pays,production,cout_production(USD/t),import_tonnes,import_USD,prix_import_USD/tonne,importFrance_tonnes,importFrance_USD,prix_importFrance_USD/tonne,export_tonnes,...,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,"27,850",0,"28,937","28,934,000","1,000",0,0,0,37,...,0,0,0,0,0,"-6,727","64,000","63,514","10,058",363
1,Afrique du Sud,"1,658,159","2,017","480,346","410,502,000",855,0,"2,000","25,000","58,259",...,0,0,0,0,0,"5,700","2,110,000","2,070,000","571,841",345
2,Albanie,"12,912",0,"17,515","16,481,000",941,0,0,0,0,...,0,0,0,0,0,0,"33,000","30,192","10,486",812
3,Algerie,"262,965","2,181",0,"2,000","5,000",0,0,0,0,...,0,0,0,0,0,0,"286,000","262,965","461,882","1,750"
4,Allemagne,"1,009,565","1,460","474,951","1,101,592,000","2,319","19,245","69,354,000","3,604","300,561",...,"32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829","415,700",412


### 5.4.2 du df "pays"

In [162]:
pays = pays.fillna(0.0)
pays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 0 to 234
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pays                   201 non-null    object 
 1   cout_douanes(USD)      201 non-null    float64
 2   delai_douanes(heures)  201 non-null    float64
 3   stabilite              201 non-null    object 
 4   lat                    201 non-null    float64
 5   lon                    201 non-null    float64
dtypes: float64(4), object(2)
memory usage: 11.0+ KB


In [163]:
pays.head()

,pays,cout_douanes(USD),delai_douanes(heures),stabilite,lat,lon
0,Afghanistan,750,96,-2.80,35,69
1,Afrique du Sud,668,87,-0.28,-26,28
2,Albanie,77,10,0.38,41,20
3,Algerie,409,327,-0.92,37,3
4,Allemagne,0,0,0.59,53,13


# 6 - Calcul des distances des pays avec la France dans le df "pays"
## 6.1 - Importation de la librairie "geopy"

In [164]:
import numpy as np
from geopy.point import Point
from geopy.distance import geodesic
#from geopy.geocoders import Nominatim
import mpu

In [165]:
# geo = pd.read_excel("geo.xlsx")

In [166]:
# geo["lat"] = geo["lat"].map(lambda x: "{:.5f}".format(x))
# geo["lon"] = geo["lon"].map(lambda x: "{:.5f}".format(x))

### 6.1.2 - Initialisation d'un géocodeur "Nominatim"

In [167]:
geolocator = Nominatim(user_agent = "my_app")

## 6.2 - Assignation des coordonnées GPS de la France à la variable "france_coords"

In [168]:
france_coords = pays.loc[pays["pays"].eq("France"), ["lat","lon"]].values[0]

In [169]:
france_coords

array([48.86666,  2.33333])

### 6.2.1 - assignation des valeurs aux variables "ref_lat" et "ref_lon" par la décomposition des valeurs contenues dans "france_coords"

In [170]:
ref_lat, ref_lon = france_coords

## 6.4 - Création de la colonne "distance_france(km)" calculant la distance de chaque pays avec la france

In [171]:
pays.head()

,pays,cout_douanes(USD),delai_douanes(heures),stabilite,lat,lon
0,Afghanistan,750,96,-2.80,35,69
1,Afrique du Sud,668,87,-0.28,-26,28
2,Albanie,77,10,0.38,41,20
3,Algerie,409,327,-0.92,37,3
4,Allemagne,0,0,0.59,53,13


In [172]:
pays['distance_france(km)'] = pays.apply(lambda row: geodesic(Point(row["lat"], row["lon"]), Point(ref_lat, ref_lon)).kilometers,axis=1)

In [173]:
# pays = pays.dropna(subset=["lat", "lon"])

In [174]:
pays.head()

,pays,cout_douanes(USD),delai_douanes(heures),stabilite,lat,lon,distance_france(km)
0,Afghanistan,750,96,-2.80,35,69,"5,598"
1,Afrique du Sud,668,87,-0.28,-26,28,"8,654"
2,Albanie,77,10,0.38,41,20,"1,605"
3,Algerie,409,327,-0.92,37,3,"1,338"
4,Allemagne,0,0,0.59,53,13,882


## 6.5 - Suppression des colonnes devenues inutiles

In [175]:
pays = pays.drop(["lat","lon"], axis=1)

In [176]:
pays["distance_france(km)"] = round(pays["distance_france(km)"],0)

In [177]:
pays.head()

,pays,cout_douanes(USD),delai_douanes(heures),stabilite,distance_france(km)
0,Afghanistan,750,96,-2.80,"5,598"
1,Afrique du Sud,668,87,-0.28,"8,654"
2,Albanie,77,10,0.38,"1,605"
3,Algerie,409,327,-0.92,"1,338"
4,Allemagne,0,0,0.59,882


In [178]:
#population.to_excel("population2.xlsx")
#produit_poulet.to_excel("produit_poulet2.xlsx")
#pays.to_excel("pays2.xlsx")

# 7 - Reconstituion définitive du df_final
## 7.1 - merge des df "population" et "produit_poulet"

In [179]:
df_final = pd.merge(population,produit_poulet,on="pays",how="left")

In [180]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           201 non-null    object 
 1   population                     201 non-null    int64  
 2   %_evolution_2007/2017          201 non-null    float64
 3   cout_viande_pers/an(USD)       201 non-null    float64
 4   PIB_habitant(USD)              201 non-null    float64
 5   dispo_alimentaire(kg/pers/an)  201 non-null    float64
 6   Tx_inflation                   201 non-null    float64
 7   production                     201 non-null    float64
 8   cout_production(USD/t)         201 non-null    float64
 9   import_tonnes                  201 non-null    float64
 10  import_USD                     201 non-null    float64
 11  prix_import_USD/tonne          201 non-null    float64
 12  importFrance_tonnes            201 non-null    flo

In [181]:
df_final.head()

,pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD),dispo_alimentaire(kg/pers/an),Tx_inflation,production,cout_production(USD/t),import_tonnes,...,exportFrance_tonnes,exportFrance_USD,prix_exportFrance_USD/tonne,excedent_commercial_fr_tonnes,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,emission_co2(tonnes),tonne_co2/tonne_poulet
0,Afghanistan,35643418,38,0,"2,058",2,7,"27,850",0,"28,937",...,0,0,0,0,0,"-6,727","64,000","63,514","10,058",363
1,Afrique du Sud,56641209,13,387,"13,860",36,7,"1,658,159","2,017","480,346",...,0,0,0,0,0,"5,700","2,110,000","2,070,000","571,841",345
2,Albanie,2879355,-3,439,"12,771",11,4,"12,912",0,"17,515",...,0,0,0,0,0,0,"33,000","30,192","10,486",812
3,Algerie,41136546,21,352,"11,737",6,5,"262,965","2,181",0,...,0,0,0,0,0,0,"286,000","262,965","461,882","1,750"
4,Allemagne,82624374,2,243,"53,072",11,3,"1,009,565","1,460","474,951",...,"32,492","94,163,000","2,898","13,247","24,809,000","181,293","1,492,000","913,829","415,700",412


## 7.2 - merge des df "df_final" et "pays"

In [182]:
df_final = pd.merge(df_final,pays,on="pays",how="left")

In [183]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           201 non-null    object 
 1   population                     201 non-null    int64  
 2   %_evolution_2007/2017          201 non-null    float64
 3   cout_viande_pers/an(USD)       201 non-null    float64
 4   PIB_habitant(USD)              201 non-null    float64
 5   dispo_alimentaire(kg/pers/an)  201 non-null    float64
 6   Tx_inflation                   201 non-null    float64
 7   production                     201 non-null    float64
 8   cout_production(USD/t)         201 non-null    float64
 9   import_tonnes                  201 non-null    float64
 10  import_USD                     201 non-null    float64
 11  prix_import_USD/tonne          201 non-null    float64
 12  importFrance_tonnes            201 non-null    flo

In [184]:
df_final.head(2)

,pays,population,%_evolution_2007/2017,cout_viande_pers/an(USD),PIB_habitant(USD),dispo_alimentaire(kg/pers/an),Tx_inflation,production,cout_production(USD/t),import_tonnes,...,excedent_commercial_fr_USD,variation_stock,nourriture,disponibilite_interieure,emission_co2(tonnes),tonne_co2/tonne_poulet,cout_douanes(USD),delai_douanes(heures),stabilite,distance_france(km)
0,Afghanistan,35643418,38,0,"2,058",2,7,"27,850",0,"28,937",...,0,"-6,727","64,000","63,514","10,058",363,750,96,-2.80,"5,598"
1,Afrique du Sud,56641209,13,387,"13,860",36,7,"1,658,159","2,017","480,346",...,0,"5,700","2,110,000","2,070,000","571,841",345,668,87,-0.28,"8,654"


## 7.3 - Suppression de la France, le projet étant basé sur l'export.

In [185]:
df_final = df_final.drop(df_final[df_final["pays"] == "France"].index)

In [186]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 200
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pays                           200 non-null    object 
 1   population                     200 non-null    int64  
 2   %_evolution_2007/2017          200 non-null    float64
 3   cout_viande_pers/an(USD)       200 non-null    float64
 4   PIB_habitant(USD)              200 non-null    float64
 5   dispo_alimentaire(kg/pers/an)  200 non-null    float64
 6   Tx_inflation                   200 non-null    float64
 7   production                     200 non-null    float64
 8   cout_production(USD/t)         200 non-null    float64
 9   import_tonnes                  200 non-null    float64
 10  import_USD                     200 non-null    float64
 11  prix_import_USD/tonne          200 non-null    float64
 12  importFrance_tonnes            200 non-null    float64


# 8 - Export du df_final au format .xlsx

In [187]:
#df_final.to_excel("df_finalv2.xlsx")